In [22]:
import torch 
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import numpy as np
import pandas as pd
import cv2
import os
from tqdm import tqdm
from ultralytics import YOLO

In [23]:
#Load the YOLO models
model_early = YOLO('Models/Models/best_early.pt')
model_late = YOLO('Models/Models/best_late.pt')
model_full = YOLO('Models/Models/best_full.pt')

In [24]:
def segment_root(image_path, model):
    '''Segment the cassava roots from the image using YOLO.'''
    image = cv2.imread(image_path)
    results = model(image)
    for result in results:
        for box in result.boxes.xyxy:
            x1,y1,x2,y2 = map(int, box)
            cropped_img = image[y1:y2, x1:x2]
            cropped_image = cv2.resize(cropped_img, (256,256))
            return cropped_image
    return None

In [34]:
class CassavaRootDataset(Dataset):
    def __init__(self, data, root_dir, model, transform=None):
        self.data = pd.read_csv(data)
        self.root_dir = root_dir
        self.transform = transform
        self.model = model

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        folder_name = self.data.iloc[idx]['FolderName']
        folder_path = os.path.join(self.root_dir, folder_name)

        #check if the folder exists
        if not os.path.exists(folder_path):
            raise FileNotFoundError(f'Folder not found: {folder_path}')
        
        image_files = [f for f in os.listdir(folder_path)]

        right_images = [f for f in image_files if '_R_' in f]
        left_images = [f for f in image_files if '_L_' in f]   

        if right_images:
            right_img_path = os.path.join(folder_path, right_images[0])
            right_image = segment_root(right_img_path, self.model)
        else:
            right_image = np.zeros((256,256,3), dtype=np.uint8)

        if left_images:
            left_img_path = os.path.join(folder_path, left_images[0])
            left_image = segment_root(left_img_path, self.model)
        else:
            left_image = np.zeros((256,256,3), dtype=np.uint8)

        if right_image is None:
            right_image = np.zeros((256,256,3), dtype=np.uint8)
        if left_image is None:
            left_image = np.zeros((256,256,3), dtype=np.uint8)

        combined_image = np.transpose(combined_image, (2,0,1))


        if self.transform:
            combined_image = self.transform(combined_image)

        root_volume =torch.tensor(self.data.iloc[idx]['RootVolume'], dtype=torch.float32)
        return combined_image, root_volume

In [43]:
!dir

 Volume in drive C has no label.
 Volume Serial Number is 36D5-25E8

 Directory of c:\Users\Reinhard\Documents\computer_vision_projects\root_volume_estimation_challenge

02/20/2025  04:08 PM    <DIR>          .
02/20/2025  04:08 PM    <DIR>          ..
02/17/2025  03:31 PM    <DIR>          data
02/13/2025  08:07 PM       326,977,527 data.zip
02/18/2025  04:25 PM    <DIR>          lightning_logs
02/18/2025  03:01 PM    <DIR>          merged_images
02/17/2025  03:32 PM    <DIR>          Models
02/13/2025  08:06 PM       345,692,256 Models.zip
02/18/2025  06:51 PM                 0 notebook.ipynb
02/19/2025  07:46 PM                 0 notebook_two.ipynb
02/19/2025  05:56 PM    <DIR>          runs
02/13/2025  08:04 PM             2,484 Sample_Submission.csv
02/18/2025  04:27 PM             3,638 submission.csv
02/20/2025  04:08 PM             4,045 submission201.csv
02/13/2025  08:04 PM             7,133 Test.csv
02/13/2025  08:04 PM            22,715 Train.csv
02/17/2025  03:33 PM    <DI

In [35]:
transform= transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,) * 6, (0.5,) * 6)
])

In [36]:
#Load the datasets 
train_dataset = CassavaRootDataset('Train.csv', 'data/train', model_full, transform=transform)
test_dataset = CassavaRootDataset('Test.csv', 'data/test', model_full, transform=transform) 

In [37]:
# Get the length of the dataset
len(train_dataset)

386

In [38]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [39]:
#Define Model
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(6, 32,3)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.fc1 = nn.Linear(64*62*62, 128)
        self.fc2 = nn.Linear(128, 1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2,2)
        self.dropout = nn.Dropout(0.5)

    def forward(self,x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 64*62*62)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [40]:
##Training set up
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CNNModel().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [41]:
#Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels.unsqueeze(1))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')
    

  0%|          | 0/13 [00:00<?, ?it/s]


0: 32x640 2 roots, 715.0ms
Speed: 23.7ms preprocess, 715.0ms inference, 20.3ms postprocess per image at shape (1, 3, 32, 640)

0: 32x640 1 root, 353.0ms
Speed: 2.4ms preprocess, 353.0ms inference, 2.3ms postprocess per image at shape (1, 3, 32, 640)


  0%|          | 0/13 [00:01<?, ?it/s]


UnboundLocalError: cannot access local variable 'combined_image' where it is not associated with a value